Review and comments -> Split to sentences

Sentences -> Split to word

Word -> Processed and get sentiment

For each sentence add up the sentiment of the words

For each document add up the sentiment of the sentences

In [15]:
import json_lines
import re
import json
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag
import string
import pandas as pd
import mtranslate

In [16]:
def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [17]:
df = pd.DataFrame(columns=['Content', 'Sentiment', 'Source'])
lemmatizer = WordNetLemmatizer()

#For some reason, please returns positive sentiment because it assumes 'pleasing' but normally people just asking politely
weird_sentiment_words = ['la', 'please']

#file = json.loads(open('chi.json', encoding="utf8"))
with open('fitnessfirst.json', 'rb') as f:
    for line in json_lines.reader(f):

        try:
            #Check if key exists
            sentence = line['reviewBody']
            #print(line['message'])

            print("New review")
            print('---------------------------------------')

            #Translate to English
            sentence = mtranslate.translate(sentence, 'en', 'auto')
            
            #Split a single comment into multiple sentences
            sentences = sent_tokenize(sentence)
            
            overall_sentiment = 0
            #print(sentences)

            #Sentiment for each sentence
            for s in sentences:

                s = s.lower()
                s = s.replace('fitness first', ' ')
                s = re.sub(r"\byin\b", "", s)
                s = re.sub(r"\byang\b", "", s)
                print("Sentence: ", s)
                sentiment = 0.0
                #print(s)

                tokenized = word_tokenize(s)
                #print(tokenized)
                
                new_tokenized = [x for x in tokenized if x not in weird_sentiment_words]
                tokenized = new_tokenized
                    
                #POS tag each word
                tagged = pos_tag(tokenized)
                print(tagged)
                
                previous_word = ""
                
                #Get the sentiment for each word
                for w, t in tagged:
                    
                    sentiment_added = False
                    
                    #Convert Penn Treebank POS tag to Wordnet POS tag
                    wn_tag = penn_to_wn(t)
                    
                    #If successfully converted, lemmatize using the Wordnet POS tag
                    #If not successfully converted, lemmatize without any POS tags
                    if wn_tag is None:
                        lemma = lemmatizer.lemmatize(w)
                    else:
                        lemma = lemmatizer.lemmatize(w, pos=wn_tag)
                        
                    #If can't lemmatize, set the word to be passed into SentiWordnet as the initial word, w
                    #If can lemmatize, set the word to be passed into Sentiwordnet as the lemmatized word, lemma
                    if not lemma:
                        print("Can't lemmatize: ", w)
                        final_word = w
                    else:
                        print("Lemmatized word: ", lemma)
                        final_word = lemma
                        
                    #Make cool and wow positive words
                    if (final_word in ['cool', 'wow', 'glad']) and (previous_word == 'not'):
                        print('Not is the previous word')
                        sentiment = sentiment - 0.5
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['cool', 'wow', 'glad']):
                        print(final_word, " has a score of 0.5")
                        sentiment = sentiment + 0.5
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word == 'nice') and (previous_word == 'not'):
                        print('Not is the previous word')
                        sentiment = sentiment - 0.875
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word == 'nice'):
                        print(final_word, " has a score of 0.875")
                        sentiment = sentiment + 0.875
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['terminate', 'stop', 'halt', 'cancel']) and (previous_word == 'not'):
                        sentiment = sentiment + 0.75
                        print("The word terminate, stop or halt is detected and the sentiment is -0.75")
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['terminate', 'stop', 'halt', 'cancel']):
                        sentiment = sentiment - 0.75
                        print("The word terminate, stop or halt is detected and the sentiment is -0.75")
                        previous_word = final_word
                        sentiment_added = True
                        
                    #If the Wordnet POS tag exists, use it to get the synset of the word
                    if wn_tag is None:
                        synsets = wn.synsets(final_word)
                    else:
                        synsets = wn.synsets(final_word, pos=wn_tag)
                        
                    #If no synsets found, skip the word
                    if not synsets:
                        print("Synset not found")
                        continue
                        
                    # Take the first sense of the word which is the most common
                    synset = synsets[0]
                    swn_synset = swn.senti_synset(synset.name())
                    
                    if sentiment_added == False:
                        if final_word != 'fitness' and previous_word == 'not':      
                            print(final_word, " | ", synset, " | ", swn_synset, " | ", synset.definition())
                            sentiment = sentiment - swn_synset.pos_score() - swn_synset.neg_score()
                        elif final_word != 'fitness':
                            print(final_word, " | ", synset, " | ", swn_synset, " | ", synset.definition())
                            sentiment = sentiment + swn_synset.pos_score() - swn_synset.neg_score()
                        else:
                            print("The word fitness is not considered as a sentiment")
                    
                    previous_word = final_word
                
                overall_sentiment = overall_sentiment + sentiment

            df2 = pd.DataFrame([[sentence, overall_sentiment, 'Facebook Review Fitness First']],columns=['Content', 'Sentiment', 'Source'])
                    
            df = pd.concat([df2,df], ignore_index=True)
                    
            # sum greater than 0 => positive sentiment
            if overall_sentiment >= 0:
                print("This document has a positive sentiment: " + str(overall_sentiment))
            elif overall_sentiment < 0:
                print("This document has a negative sentiment: " + str(overall_sentiment))
            print()
        except:
            continue

New review
---------------------------------------
Sentence:  best.
[('best', 'JJS'), ('.', '.')]
Lemmatized word:  best
best  |  Synset('best.a.01')  |  <best.a.01: PosScore=0.75 NegScore=0.0>  |  (superlative of `good') having the most positive qualities
Lemmatized word:  .
Synset not found
This document has a positive sentiment: 0.75

New review
---------------------------------------
Sentence:   's management service getting from bad to worst for my 15 years journey with them especially i am terminating my membership now.
[("'s", 'POS'), ('management', 'NN'), ('service', 'NN'), ('getting', 'VBG'), ('from', 'IN'), ('bad', 'JJ'), ('to', 'TO'), ('worst', 'VB'), ('for', 'IN'), ('my', 'PRP$'), ('15', 'CD'), ('years', 'NNS'), ('journey', 'NN'), ('with', 'IN'), ('them', 'PRP'), ('especially', 'RB'), ('i', 'VBP'), ('am', 'VBP'), ('terminating', 'VBG'), ('my', 'PRP$'), ('membership', 'NN'), ('now', 'RB'), ('.', '.')]
Lemmatized word:  's
Synset not found
Lemmatized word:  management
managem

In [18]:
df

,Content,Sentiment,Source
0,I’ve been a member of FF for many years. Of la...,1.625,Facebook Review Fitness First
1,Poor services and i had been cheated by u! As...,-1.625,Facebook Review Fitness First
2,Hi I have been a fitness first member for more...,-0.750,Facebook Review Fitness First
3,Setia City mall class schedule in app and webs...,1.375,Facebook Review Fitness First
4,I think FF members like to roll in the sweat o...,-0.625,Facebook Review Fitness First
5,Relocation of FF IOI Mall has made me and many...,-0.125,Facebook Review Fitness First
6,Poor service. Couldn’t even provide basic nece...,-0.875,Facebook Review Fitness First
7,For your information I am used to attend Brian...,0.000,Facebook Review Fitness First
8,"Within 3 months plus I joined the FF, the staf...",-2.625,Facebook Review Fitness First
9,"I have always prefer FF than other clubs, prob...",-0.125,Facebook Review Fitness First


In [19]:
df3 = pd.DataFrame(columns=['Content', 'Sentiment', 'Source'])
lemmatizer = WordNetLemmatizer()

file = json.load(open('fitnessfirst6months.json', encoding="utf8"))
for line in file['data']:
    
    try:
        #Check if key exists
        line['comments']
        #print(line['message'])
        
        for comments in line['comments']['data']:
            print("New comment")
            print('---------------------------------------')
            sentence = comments['message']
            
            #Translate to English
            sentence = mtranslate.translate(sentence, 'en', 'auto')
            
            #Split a single comment into multiple sentences
            sentences = sent_tokenize(sentence)
            #print(sentences)
            
            if len(sentences) == 0:
                continue
                
            overall_sentiment = 0
            
            #Sentiment for each sentence
            for s in sentences:
                    
                s = s.lower()
                s = s.replace('fitness first', ' ')
                s = re.sub(r"\byin\b", "", s)
                s = re.sub(r"\byang\b", "", s)
                
                print("Sentence: ", s)
                sentiment = 0.0
                #print(s)
                
                tokenized = word_tokenize(s)
                #print(tokenized)
                
                new_tokenized = [x for x in tokenized if x not in weird_sentiment_words]
                tokenized = new_tokenized
                    
                #POS tag each word
                tagged = pos_tag(tokenized)
                print(tagged)
                
                previous_word = ""
                
                #Get the sentiment for each word
                for w, t in tagged:
                    
                    sentiment_added = False
                    
                    #Convert Penn Treebank POS tag to Wordnet POS tag
                    wn_tag = penn_to_wn(t)
                    
                    #If successfully converted, lemmatize using the Wordnet POS tag
                    #If not successfully converted, lemmatize without any POS tags
                    if wn_tag is None:
                        lemma = lemmatizer.lemmatize(w)
                    else:
                        lemma = lemmatizer.lemmatize(w, pos=wn_tag)
                        
                    #If can't lemmatize, set the word to be passed into SentiWordnet as the initial word, w
                    #If can lemmatize, set the word to be passed into Sentiwordnet as the lemmatized word, lemma
                    if not lemma:
                        print("Can't lemmatize: ", w)
                        final_word = w
                    else:
                        print("Lemmatized word: ", lemma)
                        final_word = lemma
                        
                    #Make cool and wow positive words
                    if (final_word in ['cool', 'wow', 'glad']) and (previous_word == 'not'):
                        print('Not is the previous word')
                        sentiment = sentiment - 0.5
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['cool', 'wow', 'glad']):
                        print(final_word, " has a score of 0.5")
                        sentiment = sentiment + 0.5
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word == 'nice') and (previous_word == 'not'):
                        print('Not is the previous word')
                        sentiment = sentiment - 0.875
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word == 'nice'):
                        print(final_word, " has a score of 0.875")
                        sentiment = sentiment + 0.875
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['terminate', 'stop', 'halt', 'cancel']) and (previous_word == 'not'):
                        sentiment = sentiment + 0.75
                        print("The word terminate, stop or halt is detected and the sentiment is -0.75")
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['terminate', 'stop', 'halt', 'cancel']):
                        sentiment = sentiment - 0.75
                        print("The word terminate, stop or halt is detected and the sentiment is -0.75")
                        previous_word = final_word
                        sentiment_added = True
                        
                    #If the Wordnet POS tag exists, use it to get the synset of the word
                    if wn_tag is None:
                        synsets = wn.synsets(final_word)
                    else:
                        synsets = wn.synsets(final_word, pos=wn_tag)
                        
                    #If no synsets found, skip the word
                    if not synsets:
                        print("Synset not found")
                        continue
                        
                    # Take the first sense of the word which is the most common
                    synset = synsets[0]
                    swn_synset = swn.senti_synset(synset.name())
                    
                    if sentiment_added == False:
                        if final_word != 'fitness' and previous_word == 'not':      
                            print(final_word, " | ", synset, " | ", swn_synset, " | ", synset.definition())
                            sentiment = sentiment - swn_synset.pos_score() - swn_synset.neg_score()
                        elif final_word != 'fitness':
                            print(final_word, " | ", synset, " | ", swn_synset, " | ", synset.definition())
                            sentiment = sentiment + swn_synset.pos_score() - swn_synset.neg_score()
                        else:
                            print("The word fitness is not considered as a sentiment")
                    
                    previous_word = final_word
                
                overall_sentiment = overall_sentiment + sentiment

            df2 = pd.DataFrame([[sentence, overall_sentiment, 'Facebook Comment Fitness First']],columns=['Content', 'Sentiment', 'Source'])
                    
            df3 = pd.concat([df2,df3], ignore_index=True)
                    
            # sum greater than 0 => positive sentiment
            if overall_sentiment >= 0:
                print("This document has a positive sentiment: " + str(overall_sentiment))
            elif overall_sentiment < 0:
                print("This document has a negative sentiment: " + str(overall_sentiment))
        print()
    except:
        continue

New comment
---------------------------------------
Sentence:  farah zuhanazullkefle pnjm card nk msk ni..
[('farah', 'NN'), ('zuhanazullkefle', 'NN'), ('pnjm', 'FW'), ('card', 'NN'), ('nk', 'NN'), ('msk', 'NN'), ('ni..', 'NN')]
Lemmatized word:  farah
Synset not found
Lemmatized word:  zuhanazullkefle
Synset not found
Lemmatized word:  pnjm
Synset not found
Lemmatized word:  card
card  |  Synset('card.n.01')  |  <card.n.01: PosScore=0.0 NegScore=0.0>  |  one of a set of small pieces of stiff paper marked in various ways and used for playing games or for telling fortunes
Lemmatized word:  nk
Synset not found
Lemmatized word:  msk
Synset not found
Lemmatized word:  ni..
Synset not found
This document has a positive sentiment: 0.0
New comment
---------------------------------------
Sentence:  ben lucas your bags
[('ben', 'NN'), ('lucas', 'VBZ'), ('your', 'PRP$'), ('bags', 'NNS')]
Lemmatized word:  ben
ben  |  Synset('ben.n.01')  |  <ben.n.01: PosScore=0.125 NegScore=0.0>  |  a mountain o

In [20]:
final_df = pd.concat([df,df3], ignore_index=True)
final_df = final_df[final_df['Sentiment'] != 0]

In [21]:
len(final_df[final_df['Sentiment'] < 0])

68

In [24]:
len(final_df[final_df['Sentiment'] > 0])

89

In [22]:
final_df

,Content,Sentiment,Source
0,I’ve been a member of FF for many years. Of la...,1.625,Facebook Review Fitness First
1,Poor services and i had been cheated by u! As...,-1.625,Facebook Review Fitness First
2,Hi I have been a fitness first member for more...,-0.750,Facebook Review Fitness First
3,Setia City mall class schedule in app and webs...,1.375,Facebook Review Fitness First
4,I think FF members like to roll in the sweat o...,-0.625,Facebook Review Fitness First
5,Relocation of FF IOI Mall has made me and many...,-0.125,Facebook Review Fitness First
6,Poor service. Couldn’t even provide basic nece...,-0.875,Facebook Review Fitness First
8,"Within 3 months plus I joined the FF, the staf...",-2.625,Facebook Review Fitness First
9,"I have always prefer FF than other clubs, prob...",-0.125,Facebook Review Fitness First
10,The facilities provided are not up to the stan...,-3.500,Facebook Review Fitness First


In [23]:
final_df.to_json('fitness_first_sentiment.json', orient='records')